<a href="https://colab.research.google.com/github/jieunseo02/portfolio/blob/main/MLkaggle_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2025-hallym-mlp-project-3/sample_submission.csv
/kaggle/input/2025-hallym-mlp-project-3/train.csv
/kaggle/input/2025-hallym-mlp-project-3/test.csv


In [ ]:
df_tr = pd.read_csv('/kaggle/input/2025-hallym-mlp-project-3/train.csv', index_col='ID')
df_te = pd.read_csv('/kaggle/input/2025-hallym-mlp-project-3/test.csv', index_col='ID')
df_submission = pd.read_csv('/kaggle/input/2025-hallym-mlp-project-3/sample_submission.csv', index_col='ID')

In [ ]:
print(df_tr.shape)
df_tr.head()

(17519, 13)


,year,month,day,days,hour,ta,wd,ws,rn_day,rn_hr1,hm,ta_chi,demand
ID,,,,,,,,,,,,,
0,2023,1,1,1,1,-7.60,193.70,1.10,0.0,0.0,58.15,-8.30,644.0
1,2023,1,1,1,2,-7.85,76.70,0.75,0.0,0.0,57.50,-8.75,632.0
2,2023,1,1,1,3,-9.45,148.45,0.45,0.0,0.0,66.00,-9.45,610.0
3,2023,1,1,1,4,-9.55,141.10,0.40,0.0,0.0,66.15,-9.70,595.5
4,2023,1,1,1,5,-9.45,162.15,1.00,0.0,0.0,69.95,-9.90,573.5


In [ ]:
print(df_te.shape)
df_te.head()

(8760, 12)


,year,month,day,days,hour,ta,wd,ws,rn_day,rn_hr1,hm,ta_chi
ID,,,,,,,,,,,,
0,2025,1,1,1,0,-2.2,100.15,0.35,0.0,0.0,75.75,-1.90
1,2025,1,1,1,1,-1.7,128.90,0.70,0.0,0.0,74.85,-1.45
2,2025,1,1,1,2,-2.0,211.25,0.20,0.0,0.0,76.15,-1.55
3,2025,1,1,1,3,-2.2,321.80,0.15,0.0,0.0,79.05,-1.75
4,2025,1,1,1,4,-1.5,324.05,0.20,0.0,0.0,78.90,-1.55


In [ ]:
print(df_submission.shape)
df_submission.head()

(8760, 1)


,demand
ID,
0,574.306512
1,546.912154
2,506.301659
3,482.623444
4,472.110690


In [ ]:
## 훈련데이터
X_tr = df_tr.drop(columns='demand')
y_tr = df_tr['demand']

## 시험데이터
X_te = df_te.copy()

2. 데이터 전처리 및 회귀 모델 개발

In [ ]:
df_tr.describe()

,year,month,day,days,hour,ta,wd,ws,rn_day,rn_hr1,hm,ta_chi,demand
count,17519.000000,17519.000000,17519.000000,17519.000000,17519.000000,17519.000000,17519.000000,17519.000000,17519.000000,17519.000000,17519.000000,17519.000000,17519.000000
mean,2023.500029,6.526343,15.721388,183.010389,11.500656,12.635793,196.445068,-3.100111,-12.511781,-16.305226,62.474062,13.583133,232.185342
std,0.500014,3.447795,8.796046,105.363070,6.922036,76.555363,203.058588,185.141936,337.836951,369.921910,79.051670,11.844822,181.743020
min,2023.000000,1.000000,1.000000,1.000000,0.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-22.250000,22.000000
25%,2023.000000,4.000000,8.000000,92.000000,6.000000,4.900000,148.750000,0.500000,0.000000,0.000000,47.400000,4.550000,90.000000
50%,2024.000000,7.000000,16.000000,183.000000,12.000000,14.450000,200.600000,1.050000,0.000000,0.000000,63.800000,14.600000,152.000000
75%,2024.000000,10.000000,23.000000,274.000000,17.500000,22.850000,254.475000,1.700000,0.000000,0.000000,80.550000,23.900000,355.500000
max,2024.000000,12.000000,31.000000,365.000000,23.000000,36.950000,360.000000,5.550000,135.500000,13.750000,99.600000,35.450000,838.500000


In [ ]:
df = df_tr.copy()
df[df == -9999] = np.nan

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import make_pipeline

In [ ]:
numerical_columns = X_tr.select_dtypes(include=["int64", "float64"]).columns.tolist()
preprocessor = ColumnTransformer(
    transformers=[("num", StandardScaler(), numerical_columns)]
)

In [ ]:
one_hot_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

In [ ]:
ridge_pipeline = make_pipeline(preprocessor, RidgeCV())
gbr_pipeline = make_pipeline(preprocessor, HistGradientBoostingRegressor())

In [ ]:
model = ridge_pipeline.fit(X_tr, y_tr)
model = gbr_pipeline.fit(X_tr, y_tr)

3. 제출용 CSV 파일 만들기

In [ ]:
p_te = model.predict(X_te)

In [ ]:
df_submission.loc[:,'demand'] = p_te

In [ ]:
df_submission.to_csv('/kaggle/working/submission.csv')